# BentoML PyTorch TLDR Tutorial

Link to source code: https://github.com/bentoml/gallery/tree/main/TLDR/pytorch_seq2seq/

Install required dependencies:

In [1]:
!python3 -m pip install --quiet -r requirements.txt

distutils: /Users/spence/Library/Python/3.8/include/python3.8/UNKNOWN
sysconfig: /Users/spence/Library/Python/3.8/include/UNKNOWN
user = True
home = None
root = None
prefix = None
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


Import required libraries:

## Download and Extract Reddit TL;DR

In [2]:
import requests
from os.path import exists
import zipfile
import json
from tqdm import trange, tqdm
import pickle

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
import pandas as pd

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
dataset_path = 'dataset/corpus-webis-tldr-17.zip'

if not exists(dataset_path):
    r = requests.get('https://zenodo.org/record/1043504/files/corpus-webis-tldr-17.zip?download=1')
    with open(dataset_path, 'wb') as f:
        f.write(r.content)
    with zipfile.ZipFile(dataset_path, 'r') as z:
        z.extractall('dataset/')

In [4]:
df = pd.read_json('dataset/corpus-webis-tldr-17.json', \
                  lines=True, chunksize=1000000)

## Begin processing data

In [5]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: 'SOS', 1: 'EOS'}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r'([.!?])', r' \1', s)
    s = re.sub(r'[^a-zA-Z.!?]+', r' ', s)
    return s

In [6]:
def readLangs(lang1, lang2, reverse=False):
    print('Reading lines...')

    lines = next(df)

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l] for l in \
             tqdm(zip(lines['content'], lines['summary']))]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [7]:
MAX_LENGTH = 260

eng_prefixes = (
    'i am ', 'i m ',
    'he is', 'he s ',
    'she is', 'she s ',
    'you are', 'you re ',
    'we are', 'we re ',
    'they are', 'they re ',
    
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [8]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print('Read %s sentence pairs' % len(pairs))
    pairs = filterPairs(pairs)
    print('Trimmed to %s sentence pairs' % len(pairs))
    print('Counting words...')
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print('Counted words:')
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('content', 'summary')

with open('models/vectors.pkl', 'w') as f:
    pickle.dump({
        'input_lang': input_lang,
        'output_lang': output_lang,
    }, f)

Reading lines...


1000000it [06:22, 2616.12it/s]


Read 1000000 sentence pairs
Trimmed to 27733 sentence pairs
Counting words...
Counted words:
content 58613
summary 22268


## Define the model

First let's define a simple PyTorch network

In [9]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [10]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [11]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

## Training and Saving the model

Then we define a simple PyTorch network and some helper functions

In [12]:
def indexesFromSentence(lang, sentence):
    indices = []
    for word in sentence.split(' '):
        if word in lang.word2index:
            # lang.addWord(word)
            indices.append(lang.word2index[word])
    return indices


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [13]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [14]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [15]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))
            
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

### Cross Validation

We can do some cross validation and the results can be saved with the model as metadata


In [16]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [17]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [18]:
hidden_size = 256

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder, decoder, 75000, print_every=1000)

53m 47s (- 3981m 5s) (1000 1%) 4.5772
105m 1s (- 3833m 41s) (2000 2%) 3.6802
166m 58s (- 4007m 35s) (3000 4%) 3.7354
225m 44s (- 4006m 55s) (4000 5%) 3.8387
276m 6s (- 3865m 35s) (5000 6%) 3.7919
326m 30s (- 3754m 50s) (6000 8%) 3.5912
379m 53s (- 3690m 22s) (7000 9%) 3.5397
430m 38s (- 3606m 41s) (8000 10%) 3.5652
483m 39s (- 3546m 51s) (9000 12%) 3.6746
534m 42s (- 3475m 37s) (10000 13%) 3.5899
589m 9s (- 3427m 51s) (11000 14%) 3.4993
644m 26s (- 3383m 19s) (12000 16%) 3.3854
699m 10s (- 3334m 31s) (13000 17%) 3.5558
756m 8s (- 3294m 36s) (14000 18%) 3.5011
819m 17s (- 3277m 11s) (15000 20%) 3.6550
879m 30s (- 3243m 9s) (16000 21%) 3.4463
936m 1s (- 3193m 29s) (17000 22%) 3.4391
990m 26s (- 3136m 23s) (18000 24%) 3.5100
1042m 8s (- 3071m 35s) (19000 25%) 3.4663
1096m 32s (- 3015m 29s) (20000 26%) 3.3888
1147m 28s (- 2950m 39s) (21000 28%) 3.3953
1199m 43s (- 2890m 15s) (22000 29%) 3.5080
1249m 29s (- 2824m 55s) (23000 30%) 3.4492
1301m 3s (- 2764m 45s) (24000 32%) 3.5475
1354m 12s (-

In [19]:
evaluateRandomly(encoder, decoder)

> its hypocritical govt collects and releases personal information for political gain i .e . irs scandal . yet snowden collects and releases govt information for patriotic reasons and he is a terrorist traitor . snowden has only done to the feds that they continually do to us every day . i am amazed that more people are not sickened by the over reaching power of the federal govt which all started under bush but obama has only made it worse . cispa nsa obamacare irs benghazi fast and furious etc . all examples of how our government is out of control with zero accountability . just say no to big government ! it does not matter if you are democrat republican progressive or whatever when it is not your guy in the white house they will still have to much power . edit 
= they are all liars and need to be thrown out all of them . . .
< i m not to a to in the . . <EOS>

> most of the content this subreddit brings to my front page is bullying . the content itself and the act of trying to cringe

### saving the model

In [21]:
import bentoml

tag_encoder = bentoml.pytorch.save(
    'pytorch_seq2seq_encoder',
    encoder,
)

tag_decoder = bentoml.pytorch.save(
    'pytorch_seq2seq_decoder',
    decoder,
)

[20:51:51] INFO     [boot] Successfully saved                                                
                    Model(tag='pytorch_seq2seq_encoder:7pirkkus2gfh7chj', path='/Users/spence/be
                    ntoml/models/pytorch_seq2seq_encoder/7pirkkus2gfh7chj/')

           INFO     [boot] Successfully saved                                                
                    Model(tag='pytorch_seq2seq_decoder:7p4qygus2gfh7chj', path='/Users/spence/be
                    ntoml/models/pytorch_seq2seq_decoder/7p4qygus2gfh7chj/')

## Create a BentoML Service for serving the model

Note: using `%%writefile` here because `bentoml.Service` instance must be created in a separate `.py` file

Even though we have only one model, we can create as many api endpoints as we want. Here we create two end points `summarize` and `summarize_batch`

In [29]:
%%writefile service.py

import typing as t

import bentoml
from bentoml.io import Text


def unicodeToAscii(s):
    return "".join(
        c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn"
    )


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s


encoder_runner = bentoml.pytorch.load_runner("pytorch_seq2seq_encoder:latest")
decoder_runner = bentoml.pytorch.load_runner("pytorch_seq2seq_decoder:latest")

svc = bentoml.Service(
    name="pytorch_seq2seq",
    runners=[
        encoder_runner,
        decoder_runner,
    ],
)


@svc.api(input=Text(), output=Text())
async def summarize(input_sentence: str) -> str:
    input_sentence = normalizeString(input_sentence)
    enc_sentence = await encoder.async_run(input_sentence)
    res = await decoder.async_run(enc_sentence)
    return res["generated_text"]


@svc.api(input=Text(), output=Text())
async def summarize_batch(input_arr: [str]) -> [str]:
    input_arr = list(map(normalizeString, input_arr))
    results = []
    for input_sentence in input_arr:
        enc_arr = await encoder.async_run(input_arr)
        res = await decoder.async_run(enc_arr)
        results.append(res["generated_text"])
    return results


Overwriting service.py


Start a dev model server to test out the service defined above

In [30]:
!python3 -m bentoml serve service.py:svc

[20:52:45] INFO     [boot] Starting development BentoServer from                
                    'service.py:svc'                                            
[20:52:52] INFO     [dev_api_server] Service imported from source:              
                    bentoml.Service(name='pytorch_seq2seq',                   
                    import_str='service:svc', working_dir='/Users/spence/Documen
                    ts/GitHub/gallery/pytorch_seq2seq')                         
[20:52:52] INFO     [dev_api_server] Started server process [12124]             
           INFO     [dev_api_server] Waiting for application startup.           
           INFO     [dev_api_server] Application startup complete.              
           INFO     [dev_api_server] Uvicorn running on http://127.0.0.1:5000   
                    (Press CTRL+C to quit)                                      
^C
[20:52:56] INFO     [dev_api_server] Shutting down                              
           INFO     [dev_ap

## Build a Bento for distribution and deployment

In [31]:
bentoml.build(
    'service.py:svc',
    include=['*.py'],
    exclude=['tests/'],
    description='file:./README.md',
    python=dict(
        packages=['torch'],
    )
)

[20:53:00] INFO     [boot] Building BentoML service 'pytorch_seq2seq:eup5ynus2kfh7chj' from
                    build context '/Users/spence/Documents/GitHub/gallery/pytorch_seq2seq'

           INFO     [boot] Packing model 'pytorch_seq2seq_encoder:735m27es2gfh7chj' from        
                    '/Users/spence/bentoml/models/pytorch_seq2seq_encoder/735m27es2gfh7chj'

           INFO     [boot] Packing model 'pytorch_seq2seq_decoder:74jnn5us2gfh7chj' from        
                    '/Users/spence/bentoml/models/pytorch_seq2seq_decoder/74jnn5us2gfh7chj'

           INFO     [boot] Locking PyPI package versions..

[20:53:01] INFO     [boot] Bento build success,                                              
                    Bento(tag='pytorch_seq2seq:eup5ynus2kfh7chj') created

Bento(tag='pytorch_seq2seq:eup5ynus2kfh7chj')

Starting a dev server with the Bento build:

In [33]:
!python3 -m bentoml serve pytorch_seq2seq:latest

[20:53:17] INFO     [boot] Starting development BentoServer from                
                    'pytorch_seq2seq:latest'                                  
[20:53:24] INFO     [dev_api_server] Service loaded from Bento store:           
                    bentoml.Service(tag='pytorch_seq2seq:eup5ynus2kfh7chj', pa
                    th='/Users/spence/bentoml/bentos/pytorch_seq2seq/eup5ynus2
                    kfh7chj')                                                   
[20:53:24] INFO     [dev_api_server] Started server process [12144]             
           INFO     [dev_api_server] Waiting for application startup.           
           INFO     [dev_api_server] Application startup complete.              
           INFO     [dev_api_server] Uvicorn running on http://127.0.0.1:5000   
                    (Press CTRL+C to quit)                                      
^C
[20:53:34] INFO     [dev_api_server] Shutting down                              
           INFO     [dev_api_se